In [2]:
import os
import re
import sys
import math
import json
import requests
import pandas as pd
from glob import glob
from tqdm import tqdm

# ===== 모듈 경로 추가 & 로거 설정 =====
sys.path.append(r"C:\ESG_Project1\util")
from logger import setup_logger
logger = setup_logger(__name__)

# ----------------------------------------------------
# 🔹 경로 설정
# ----------------------------------------------------
BASE_DIR = "C:/ESG_Project1/file/"
DATA_DIR = "C:/ESG_Project1/data/"

SOLAR_DIR = os.path.join(BASE_DIR, "solar_data_file")
SOLAR_LOCATION_DIR = os.path.join(BASE_DIR, "solar_data_file/location")
KMA_DIR = os.path.join(BASE_DIR, "KMA_data_file")
WEATHER_META = os.path.join(KMA_DIR, "META_관측지점정보.csv")
MANAGED_JSON = os.path.join(DATA_DIR, "json/managed_human.json")
LOCATION_JSON = os.path.join(DATA_DIR, "json/cache_location.json")

OUT_FILE_NAME = 'train_data.csv'
OUT_CSV = os.path.join(BASE_DIR, "merge_data", OUT_FILE_NAME)

# ----------------------------------------------------
# 🔹 년도 설정
# ----------------------------------------------------
TARGET_YEARS = TARGET_YEARS = list(range(2013, 2023 + 1))

# ----------------------------------------------------
# 🔹 카카오 API 키 (반드시 입력)
# ----------------------------------------------------
KAKAO_API_KEY = "93c089f75a2730af2f15c01838e892d3"  # ⚠️ 여기에 REST API 키 입력

# ----------------------------------------------------
# 🔹 CSV 유틸
# ----------------------------------------------------
def sniff_delimiter(path):
    with open(path,"rb") as f:
        raw=f.read(2048)
    text=raw.decode("utf-8",errors="ignore")
    return "," if text.count(",") >= text.count("\t") else "\t"

def read_csv_safe(path):
    delim = sniff_delimiter(path)
    try:
        df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
        logger.info(f"📄 '{os.path.basename(path)}' 읽기 성공 (utf-8)")
        return df
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp949", delimiter=delim, index_col=False)
        logger.info(f"📄 '{os.path.basename(path)}' 읽기 성공 (cp949)")
        return df

# ----------------------------------------------------
# 🔹 문자열 정규화
# ----------------------------------------------------
def norm(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[\s\-\_\/\(\)\[\]\.]+", "", s)
    return s

# ----------------------------------------------------
# 🔹 컬럼명 탐지용 키워드
# ----------------------------------------------------
PLANT_TOKENS = ["발전소명","발전소","호기명","호기","플랜트","사업명","사업장","설비명","site","plant","facility","name"]
LOC_TOKENS   = ["위치","주소","지역","시도","시군구","행정구역","소재지","address","location","region","city","province","lat","lon","latitude","longitude"]
SRC_TOKENS   = ["에너지원","에너지","발전원","전원","연료","source","fuel","type"]

def detect_col(columns, tokens):
    cols = list(columns)
    for c in cols:
        c_norm = norm(c)
        for t in tokens:
            if norm(t) in c_norm:
                return c
    return None

# ----------------------------------------------------
# 🔹 주소 단순화
# ----------------------------------------------------
def simplify_address(addr: str) -> str:
    """
    주소에서 '도/시/군/구/읍/면/동' 또는 축약된 도 이름(전남, 경북 등)을 포함해
    최대 4단계까지만 남김.
    예: '전남 여수시 소라면 죽림리 25' -> '전남 여수시 소라면'
         '전라남도 여수시 소라면' -> '전라남도 여수시 소라면'
    """
    if not isinstance(addr, str):
        return addr

    addr = addr.strip()
    if not addr:
        return addr

    # ✅ 축약형 도 이름 (전남, 경북 등)도 포함
    short_province_pattern = r"^(전남|전북|경남|경북|충남|충북|강원|제주|서울|부산|대구|인천|광주|대전|울산|세종)"
    long_province_pattern = r"^(서울특별시|부산광역시|대구광역시|인천광역시|광주광역시|대전광역시|울산광역시|세종특별자치시|경기도|강원도|강원특별자치도|충청북도|충청남도|전라북도|전라남도|경상북도|경상남도|제주특별자치도)"

    tokens = []
    parts = re.split(r"\s+", addr)

    # 1️⃣ 첫 부분이 도(또는 축약형 도)면 추가
    if re.match(long_province_pattern, parts[0]):
        tokens.append(parts[0])
        parts = parts[1:]
    elif re.match(short_province_pattern, parts[0]):
        tokens.append(parts[0])
        parts = parts[1:]

    # 2️⃣ 나머지에서 시, 군, 구, 읍, 면, 동 순으로 최대 3~4개까지 추출
    for p in parts:
        if re.search(r"(시|군|구|읍|면|동)$", p):
            tokens.append(p)
        if len(tokens) >= 3:
            break

    # 3️⃣ 만약 아무 것도 못 잡았으면 앞의 3단어만 반환
    if not tokens:
        return " ".join(addr.split()[:3])

    return " ".join(tokens)

# ----------------------------------------------------
# 🔹 Haversine 거리 계산
# ----------------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# ----------------------------------------------------
# 🔹 카카오 API (캐싱 지원)
# ----------------------------------------------------
CACHE_PATH = os.path.join(DATA_DIR, "json/cache_latlon.json")
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        CACHE = json.load(f)
else:
    CACHE = {}

def get_latlon_from_kakao(address: str):
    """카카오 API를 이용해 주소 → 위도/경도. 캐시 사용."""
    if not isinstance(address, str) or not address.strip():
        return None, None

    addr_key = address.strip()
    if addr_key in CACHE:
        return CACHE[addr_key]["lat"], CACHE[addr_key]["lon"]

    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": addr_key}

    try:
        resp = requests.get(url, headers=headers, params=params, timeout=5)
        data = resp.json()
        if data.get("documents"):
            doc = data["documents"][0]
            lat, lon = float(doc["y"]), float(doc["x"])
            CACHE[addr_key] = {"lat": lat, "lon": lon}

            # 중간 저장 (안전)
            with open(CACHE_PATH, "w", encoding="utf-8") as f:
                json.dump(CACHE, f, ensure_ascii=False, indent=2)
            return lat, lon
    except Exception as e:
        logger.info(f"⚠️ 카카오 API 실패: {address} → {e}")
    return None, None

# ----------------------------------------------------
# 🔹 발전소 위치 데이터 통합
# ----------------------------------------------------
rows = []
files = sorted(glob(os.path.join(SOLAR_LOCATION_DIR, "**", "*.csv"), recursive=True))
for f in files:
    try:
        df = read_csv_safe(f)
        cols = list(df.columns)
        plant_col = detect_col(cols, PLANT_TOKENS)
        loc_col   = detect_col(cols, LOC_TOKENS)
        src_col   = detect_col(cols, SRC_TOKENS)

        if not plant_col or not loc_col:
            logger.info(f"⚠️ 주요 컬럼 누락: {f}")
            continue

        if src_col not in df.columns:
            df["에너지원"] = "태양광"
            src_col = "에너지원"

        sub = df[[plant_col, src_col, loc_col]].copy()
        sub.columns = ["발전소명", "에너지원", "위치"]
        sub["위치"] = sub["위치"].astype(str).apply(simplify_address)
        sub["source_file"] = os.path.basename(f)
        rows.append(sub)
    except Exception as e:
        logger.info(f"❌ 파일 오류: {f} → {e}")

if not rows:
    logger.info("❌ 발전소 데이터 추출 실패")
    exit()

result = pd.concat(rows, ignore_index=True)
logger.info(f"✅ 발전소 데이터 통합 완료 ({len(result)}개)")

# ----------------------------------------------------
# 🔹 META 기상관측소 불러오기
# ----------------------------------------------------
meta_df = pd.read_csv(WEATHER_META, encoding="cp949")
meta_df = meta_df.dropna(subset=["위도", "경도"])

# ----------------------------------------------------
# 🔹 카카오 API로 위경도 + 최근접 지점 찾기
# ----------------------------------------------------
lat_list, lon_list, near_stn, near_stn_name = [], [], [], []

for i, row in result.iterrows():
    addr = row["위치"]
    lat, lon = get_latlon_from_kakao(addr)
    lat_list.append(lat)
    lon_list.append(lon)

    if lat is not None and lon is not None:
        meta_df["dist"] = meta_df.apply(
            lambda x: haversine(lat, lon, x["위도"], x["경도"]), axis=1
        )
        nearest = meta_df.loc[meta_df["dist"].idxmin()]
        near_stn.append(int(nearest["지점"]))
        near_stn_name.append(nearest["지점명"])
    else:
        near_stn.append(None)
        near_stn_name.append(None)

    if i % 10 == 0:
        logger.info(f"📍 진행중 {i}/{len(result)}")

# ----------------------------------------------------
# 🔹 결과 저장
# ----------------------------------------------------
result["위도"] = lat_list
result["경도"] = lon_list
result["지점"] = near_stn
result["지점명"] = near_stn_name

# ✅ 정수형 변환 (None 허용)
result["지점"] = result["지점"].astype("Int64")

result.to_json(LOCATION_JSON, orient="records", force_ascii=False, indent=2)

# 캐시 최종 저장
with open(CACHE_PATH, "w", encoding="utf-8") as f:
    json.dump(CACHE, f, ensure_ascii=False, indent=2)

logger.info(f"📁 결과 파일: {LOCATION_JSON}")
logger.info(f"🗂 캐시 파일: {CACHE_PATH}")

# ----------------------------------------------------
# 🔹 파일명에서 [발전소명] 추출
# ----------------------------------------------------
def extract_plant_name(filename):
    match = re.search(r"\[(.*?)\]", filename)
    return match.group(1).strip() if match else None

# ----------------------------------------------------
# 🔹 시간 숫자 추출 (0-1시, 1시, 23_24시 등 전부 커버)
# ----------------------------------------------------
def extract_hour_from_col(colname):
    s = str(colname).strip().lower()
    m = re.search(r"(\d{1,2})\s*[-~_]\s*(\d{1,2})", s)
    if m:
        end = int(m.group(2))
        return end if 1 <= end <= 24 else None
    m = re.search(r"(\d{1,2})\s*시", s)
    if m:
        h = int(m.group(1))
        return h if 1 <= h <= 24 else None
    m = re.search(r"(^|\D)(\d{1,2})(?!\d)", s)
    if m:
        h = int(m.group(2))
        return h if 1 <= h <= 24 else None
    return None

# ----------------------------------------------------
# 🔹 시간 컬럼 탐지
# ----------------------------------------------------
def detect_hour_cols(df):
    candidates = []
    for c in df.columns:
        h = extract_hour_from_col(c)
        if h is not None:
            candidates.append((c, h))
    return [c for c, h in sorted(candidates, key=lambda x: x[1])]

# ----------------------------------------------------
# 🔹 단위 감지
# ----------------------------------------------------
def detect_unit(colname):
    name = colname.lower()
    if "mwh" in name:
        return "mwh"
    elif "kwh" in name:
        return "kwh"
    elif "wh" in name:
        return "wh"
    return None

def detect_unit_from_cols(cols):
    for c in cols:
        u = detect_unit(c)
        if u:
            return u
    return None

# ----------------------------------------------------
# 🔹 단위 변환 (Wh / kWh / MWh → MWh)
# ----------------------------------------------------
def convert_to_mwh(val, unit):
    try:
        val = float(val)
        if unit == "wh":
            val /= 1_000_000
        elif unit == "kwh":
            val /= 1_000
        return round(val, 4)
    except:
        return 0.0

# ----------------------------------------------------
# 🔹 주요 컬럼 탐지
# ----------------------------------------------------
def detect_columns(df):
    cols = df.columns
    name_col = next((c for c in cols if any(x in c for x in ["발전소", "발전기", "발전구분", "site", "plant"])), None)
    date_col = next((c for c in cols if any(x in c for x in ["일자", "날짜", "년월일", "date"])), None)
    ho_col = next((c for c in cols if "호기" in c or "unit" in c), None)
    return name_col, date_col, ho_col

# ----------------------------------------------------
# 🔹 메인 처리
# ----------------------------------------------------
all_frames = []

files = sorted(glob(os.path.join(SOLAR_DIR, "**", "*.csv"), recursive=True))
for f in tqdm(files, desc="CSV 처리 중"):
    try:
        df = read_csv_safe(f)

        name_col, date_col, ho_col = detect_columns(df)
        hour_cols = detect_hour_cols(df)
        plant_name = extract_plant_name(os.path.basename(f))

        # 시간 컬럼 누락
        if not hour_cols:
            msg = f"⚠️ 시간 컬럼 없음: {os.path.basename(f)}"
            print(msg)
            logger.warning(msg)
            continue

        # 날짜 컬럼 누락
        if not date_col:
            msg = f"⚠️ 날짜 컬럼 없음: {os.path.basename(f)}"
            print(msg)
            logger.warning(msg)
            continue

        # 발전소명
        if name_col:
            df["발전소명"] = df[name_col].astype(str).str.strip()
            # ⚠️ 파일명으로 대체한 게 아닐 경우 → '태양광' 없는 행 제외
            df = df[df["발전소명"].str.contains("태양광", na=False)]
        else:
            df["발전소명"] = plant_name if plant_name else "미상"

        # 발전소명 보정
        df["발전소명"] = df["발전소명"].replace({
            "영암에프원태양광b": "(군산)영암F1태양광"
        })

        # 단위
        unit = detect_unit_from_cols(hour_cols)
        if not unit:
            unit = "kwh" if not name_col else "mwh"
    
        # Melt
        df_long = df.melt(
            id_vars=[c for c in [date_col, ho_col, "발전소명"] if c in df.columns],
            value_vars=hour_cols,
            var_name="시간대",
            value_name="발전량(MWh)"
        )

        # ✅ 날짜 전처리
        df_long[date_col] = (
            df_long[date_col]
            .astype(str)
            .str.replace(r"[^0-9]", "", regex=True)
            .str[:8]
        )

        df_long[date_col] = pd.to_datetime(df_long[date_col], format="%Y%m%d", errors="coerce")
        df_long = df_long[df_long[date_col].dt.year.isin(TARGET_YEARS)]

        # ✅ 시간 추출 및 일시 생성
        hours = df_long["시간대"].apply(extract_hour_from_col).astype("Int64")
        df_long = df_long[hours.notna()].copy()
        df_long["일시"] = df_long[date_col] + pd.to_timedelta(hours - 1, unit="h")

        # ✅ 단위 변환
        df_long["발전량(MWh)"] = df_long["발전량(MWh)"].apply(lambda x: convert_to_mwh(x, unit))

        # ✅ 정리
        df_clean = df_long[["일시", "발전소명", "발전량(MWh)"]].copy()
        all_frames.append(df_clean)

    except Exception as e:
        msg = f"❌ 오류: {os.path.basename(f)} → {e}"
        print(msg)
        logger.error(msg)

# ----------------------------------------------------
# 🔹 병합 및 후처리
# ----------------------------------------------------
if not all_frames:
    raise SystemExit("❌ 처리할 데이터 없음")

merged = pd.concat(all_frames, ignore_index=True)

# ✅ 호기 합산 포함 → 발전소명 + 일시 기준 합계
final_df = merged.groupby(["발전소명", "일시"], as_index=False)["발전량(MWh)"].sum()

final_df = final_df.drop_duplicates(subset=["발전소명", "일시", "발전량(MWh)"])

# ✅ 날짜 포맷 단일화
final_df["일시"] = final_df["일시"].dt.strftime("%Y-%m-%d %H:%M:%S")

# ✅ 소수점 4자리 고정
final_df["발전량(MWh)"] = final_df["발전량(MWh)"].apply(lambda x: f"{x:.4f}")

# ✅ 컬럼 순서 조정
final_df = final_df[["일시", "발전소명", "발전량(MWh)"]]

# 1️⃣ managed_human.json 로드
try:
    with open(MANAGED_JSON, "r", encoding="utf-8") as f:
        managed_map = json.load(f)
    logger.info(f"✅ managed_human.json 로드 완료 ({len(managed_map)}개 매핑)")
except Exception as e:
    logger.error(f"❌ managed_human.json 로드 실패: {e}")
    managed_map = {}

# 2️⃣ 매핑용 컬럼 생성
#    final_df의 발전소명을 managed_human.json 기준으로 변환한 "매핑용 발전소명" 생성
final_df["매핑용발전소명"] = final_df["발전소명"].map(managed_map).fillna(final_df["발전소명"])

# 3️⃣ location 데이터 로드
try:
    with open(LOCATION_JSON, "r", encoding="utf-8") as f:
        loc_data = json.load(f)
    loc_df = pd.DataFrame(loc_data)
    logger.info(f"✅ location_all_plants_with_station.json 로드 완료 ({len(loc_df)}행)")
except Exception as e:
    logger.error(f"❌ location_all_plants_with_station.json 로드 실패: {e}")
    loc_df = pd.DataFrame(columns=["발전소명", "지점", "지점명"])

# 4️⃣ 필요한 컬럼만 유지
loc_df = loc_df[["발전소명", "지점", "지점명"]].drop_duplicates(subset=["발전소명"])

# 5️⃣ 매핑용 발전소명을 기준으로 병합
final_df = pd.merge(
    final_df,
    loc_df,
    left_on="매핑용발전소명",
    right_on="발전소명",
    how="left",
    suffixes=("", "_loc")
)

# 🔹 float으로 들어온 지점을 Int64로 변환 (NaN 허용)
if "지점" in final_df.columns:
    final_df["지점"] = pd.to_numeric(final_df["지점"], errors="coerce").astype("Int64")

# 6️⃣ 사용하지 않는 보조 컬럼 정리
final_df.drop(columns=["매핑용발전소명", "발전소명_loc"], inplace=True, errors="ignore")

# 8️⃣ 컬럼 순서 정리
final_df = final_df[["일시", "발전소명", "지점명", "지점", "발전량(MWh)"]]


# ----------------------------------------------------
# 🔹 KMA 기상 데이터 병합 (일시 포맷/컬럼명 보정 포함)
# ----------------------------------------------------
weather_files = sorted(glob(os.path.join(KMA_DIR, "OBS_ASOS_TIM_*.csv")))
weather_frames = []

for wf in tqdm(weather_files, desc="기상데이터 병합"):
    try:
        tmp = read_csv_safe(wf)

        # ✅ 날짜 포맷 통일 (초 있든 없든 처리)
        tmp["일시"] = pd.to_datetime(tmp["일시"], errors="coerce", format="mixed")

        # ✅ 주요 컬럼만 선택 및 이름 통일
        tmp = tmp[["지점","지점명","일시","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]]
        weather_frames.append(tmp)

    except Exception as e:
        logger.warning(f"⚠️ 기상파일 '{os.path.basename(wf)}' 처리 실패: {e}")

# ✅ 병합용 기상 데이터 완성
df_weather = pd.concat(weather_frames, ignore_index=True).drop_duplicates(subset=["지점","일시"])
logger.info(f"📊 기상 데이터 총 {len(df_weather)}행 병합 완료")


# ----------------------------------------------------
# 🔹 발전량 + 기상데이터 병합 함수
# ----------------------------------------------------
def merge_weather(df_power):
    # ✅ datetime 통일 (초 제거)
    df_power["일시"] = pd.to_datetime(df_power["일시"], errors="coerce")
    df_power["일시"] = df_power["일시"].dt.floor("h")  # 시 단위 정규화
    df_weather["일시"] = pd.to_datetime(df_weather["일시"], errors="coerce")
    df_weather["일시"] = df_weather["일시"].dt.floor("h")

    # ✅ 병합 (지점, 일시 기준)
    merged = pd.merge(df_power, df_weather,
                      how="left", on=["지점", "일시"],
                      suffixes=("", "_wx"))

    # ✅ 결측값 0으로 채움 (선택적)
    merged[["발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]] = \
        merged[["발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]].fillna(0)

    # ✅ 최종 컬럼 정리
    merged = merged[["일시","발전소명","지점명","지점",
                     "발전량(MWh)","기온(°C)","강수량(mm)",
                     "일조(hr)","일사(MJ/m2)"]]
    merged = merged.sort_values(["지점명", "일시"]).reset_index(drop=True)

    return merged

# ----------------------------------------------------
# 🔹 실행
# ----------------------------------------------------
final_df = merge_weather(final_df)
print(f"✅ 발전량+기상데이터 병합 완료 → 총 {len(final_df):,}행")

# 🔹 발전소별 정렬 추가
final_df = final_df.sort_values(by=["발전소명", "일시"]).reset_index(drop=True)

# 🔹 컬럼명 변경
final_df = final_df.rename(columns={
    "발전소명": "발전구분",
    "지점명": "지역",
    "지점": "지점번호",
    "발전량(MWh)": "합산발전량(MWh)"
})

# 9️⃣ 결과 저장
final_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

print(f"✅ 발전소-지점 매핑 완료 → {OUT_CSV}")
logger.info(f"✅ 발전소-지점 매핑 완료 → {OUT_CSV} (총 {len(final_df):,}행)")

[2025-10-28 11:24:15,903]✅ INFO - 📄 '한국남동발전㈜_(발전공기업 표준) 신재생에너지 사업현황_20221231.csv' 읽기 성공 (cp949)
[2025-10-28 11:24:15,906]✅ INFO - 📄 '한국남부발전(주)_태양광발전기 사양정보_20250630.csv' 읽기 성공 (cp949)
[2025-10-28 11:24:15,910]✅ INFO - 📄 '한국동서발전(주)_(발전공기업 표준)신재생에너지 사업현황_20230630.csv' 읽기 성공 (cp949)
[2025-10-28 11:24:15,913]✅ INFO - 📄 '한국서부발전(주)_신재생에너지사업현황_20231231.csv' 읽기 성공 (cp949)
[2025-10-28 11:24:15,915]✅ INFO - ✅ 발전소 데이터 통합 완료 (273개)
[2025-10-28 11:24:15,921]✅ INFO - 📍 진행중 0/273
[2025-10-28 11:24:15,934]✅ INFO - 📍 진행중 10/273
[2025-10-28 11:24:16,274]✅ INFO - 📍 진행중 20/273
[2025-10-28 11:24:16,285]✅ INFO - 📍 진행중 30/273
[2025-10-28 11:24:16,385]✅ INFO - 📍 진행중 40/273
[2025-10-28 11:24:16,399]✅ INFO - 📍 진행중 50/273
[2025-10-28 11:24:16,412]✅ INFO - 📍 진행중 60/273
[2025-10-28 11:24:16,423]✅ INFO - 📍 진행중 70/273
[2025-10-28 11:24:16,434]✅ INFO - 📍 진행중 80/273
[2025-10-28 11:24:16,524]✅ INFO - 📍 진행중 90/273
[2025-10-28 11:24:16,537]✅ INFO - 📍 진행중 100/273
[2025-10-28 11:24:16,551]✅ INFO - 📍 진행중 110/273
[2025-10-28 

CSV 처리 중:   0%|                                                                              | 0/90 [00:00<?, ?it/s]

[2025-10-28 11:24:16,863]✅ INFO - 📄 '한국남동발전㈜_(발전공기업 표준) 신재생에너지 사업현황_20221231.csv' 읽기 성공 (cp949)
⚠️ 시간 컬럼 없음: 한국남동발전㈜_(발전공기업 표준) 신재생에너지 사업현황_20221231.csv
[2025-10-28 11:24:16,863]✅ WARNING - ⚠️ 시간 컬럼 없음: 한국남동발전㈜_(발전공기업 표준) 신재생에너지 사업현황_20221231.csv
[2025-10-28 11:24:16,866]✅ INFO - 📄 '한국남부발전(주)_태양광발전기 사양정보_20250630.csv' 읽기 성공 (cp949)
⚠️ 시간 컬럼 없음: 한국남부발전(주)_태양광발전기 사양정보_20250630.csv
[2025-10-28 11:24:16,866]✅ WARNING - ⚠️ 시간 컬럼 없음: 한국남부발전(주)_태양광발전기 사양정보_20250630.csv
[2025-10-28 11:24:16,868]✅ INFO - 📄 '한국동서발전(주)_(발전공기업 표준)신재생에너지 사업현황_20230630.csv' 읽기 성공 (cp949)
⚠️ 시간 컬럼 없음: 한국동서발전(주)_(발전공기업 표준)신재생에너지 사업현황_20230630.csv
[2025-10-28 11:24:16,869]✅ WARNING - ⚠️ 시간 컬럼 없음: 한국동서발전(주)_(발전공기업 표준)신재생에너지 사업현황_20230630.csv
[2025-10-28 11:24:16,870]✅ INFO - 📄 '한국서부발전(주)_신재생에너지사업현황_20231231.csv' 읽기 성공 (cp949)
⚠️ 시간 컬럼 없음: 한국서부발전(주)_신재생에너지사업현황_20231231.csv
[2025-10-28 11:24:16,871]✅ WARNING - ⚠️ 시간 컬럼 없음: 한국서부발전(주)_신재생에너지사업현황_20231231.csv
[2025-10-28 11:24:16,875]✅ INFO - 📄 '남동발전량_2022_01.csv' 읽기 성공 (utf

C:\Users\c\AppData\Local\Temp\ipykernel_31128\2808167551.py:54: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_31128\2808167551.py:54: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
CSV 처리 중:   7%|████▋                                                                 | 6/90 [00:00<00:02, 35.10it/s]

[2025-10-28 11:24:17,037]✅ INFO - 📄 '남동발전량_2022_03.csv' 읽기 성공 (utf-8)


...

[2025-10-28 11:24:46,956]✅ INFO - 📄 'OBS_ASOS_TIM_2024.csv' 읽기 성공 (cp949)


기상데이터 병합:  92%|████████████████████████████████████████████████████████████     | 12/13 [00:09<00:00,  1.24it/s]

[2025-10-28 11:24:47,651]✅ INFO - 📄 'OBS_ASOS_TIM_2025.csv' 읽기 성공 (cp949)


기상데이터 병합: 100%|█████████████████████████████████████████████████████████████████| 13/13 [00:10<00:00,  1.26it/s]


[2025-10-28 11:24:49,280]✅ INFO - 📊 기상 데이터 총 10600217행 병합 완료
